# Deep Q-Learning - Space Invaders
Train an agent to play Space Invaders using [Deep Reinforcement Learning](https://deepmind.google/discover/blog/deep-reinforcement-learning/)

# Outline
- [ 1 - Import Packages <img align="Right" src="./images/space_invaders.png" width = 60% >](#1)
- [ 2 - Hyperparameters](#2)
- [ 3 - The Game (Space Invaders)](#3)
  - [ 3.1 Action Space](#3.1)
  - [ 3.2 Observation Space](#3.2)
  - [ 3.3 Rewards](#3.3)
  - [ 3.4 Episode Termination](#3.4)
- [ 4 - Load the game using Selenium](#4)
- [ 5 - Game State](#5)
- [ 6 - Deep Q-Learning](#6)
  - [ 6.1 Target Network](#6.1)
  - [ 6.2 Experience Replay](#6.2)

<a name="1"></a>
## 1 - Import Packages

We'll make use of the following packages:
- `selenium` is a package for Selenium Python bindings provides a simple API to write functional/acceptance tests using Selenium WebDriver.
- `PIL` is a package from Pillow, the Python Imaging Library adds image processing capabilities.
- `OpenCV` (Open Source Computer Vision Library) is an open source computer vision and machine learning software library.
- `numpy` is the fundamental package for scientific computing in Python.
- `TensorFlow` is an end-to-end open source platform for machine learning
- `TensorFlow.Keras` is the high-level API of the TensorFlow platform. It provides an approachable, highly-productive interface for solving machine learning (ML) problems, with a focus on modern deep learning.

In [ ]:
%load_ext autoreload
%autoreload 2

In [42]:
from config import Config
from classes.game import Game
from classes.game_state import Game_State

import numpy as np
import cv2

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam

<a name="2"></a>
## 2 - Hyperparameters

Agent hyperparameters

In [ ]:
BUFFER_SIZE = 100_000     # size of memory buffer
GAMMA = 0.995             # discount factor
ALPHA = 1e-3              # learning rate
NUM_STEPS_FOR_UPDATE = 4  # perform a learning update every C time steps

<a name="3"></a>
## 3 - Space Invaders
In this notebook we will be using [free80sarcade Space Invaders](https://www.free80sarcade.com/spaceinvaders.php) web version.

Our goal is develop an AI agent that is able to learn how to play the popular game Space Invaders from scratch. To do it, we implement a Deep Reinforcement Learning algorithm using both Keras on top of Tensorflow.

<a name="3.1"></a>
### 3.1 Action Space

The agent has four discrete actions available:

* Do nothing.
* Move right.
* Move left.
* Fire weapon.

Each action has a corresponding numerical value:

```python
Do nothing = 0
Fire right engine = 1
Fire main engine = 2
Fire left engine = 3
```

<a name="3.2"></a>
### 3.2 Observation Space

<img src="./images/observation_space.png" width="268" height="158" ><br />

Screen coordinate: x = 0, y = 58, cx = 268, cy = 158

BW Image size: 268 * 158 = 42344

<a name="3.3"></a>
### 3.3 Rewards

After every step, a reward is granted. The total reward of an episode is the sum of the rewards for all the steps within that episode.

For each step, the reward:
- is increased by the difference between the previous and current scores.
- is decreased by 0.03 each time the weapon is fired
- is decreased by 100 when a live is lost

<a name="3.4"></a>
### 3.4 Episode Termination

An episode ends (i.e the environment enters a terminal state) if:

* The number of lives reach 0.


<a name="4"></a>
## 4 - Load Space Invaders using Selenium

We start by loading the `Space Invaders` emulation from the [free80sarcade](https://www.free80sarcade.com/spaceinvaders.php) website by using the `Selenium` webdriver.

In [33]:
config = Config()

# Create an instance of the Service object
service = Service(executable_path=config.chrome_driver)
# Start Chrome using the service keyword
driver = webdriver.Chrome(service=service)
# Load the site
driver.get(config.game_url)

wait = WebDriverWait(driver, 10)

# Cookie consent
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='fc-button fc-cta-consent fc-primary-button']"))).click()

# Find canvas and create Game
canvas = wait.until(EC.element_to_be_clickable((By.ID, 'videoCanvas')))

# Create the game
game = Game(driver, canvas)


<a name="5"></a>
## 5 - Game State



To build our neural network later on we need to know the size of the state vector and the number of valid actions.

In [32]:
state_size = Game_State.shape
num_actions = Game_State.num_actions

print('State Shape:', state_size)
print('Number of actions:', num_actions)

State Shape: (158, 268)
Number of actions: 3


In [40]:
game.start()

game_state = Game_State(game.get_image_bw())

cv2.imshow('Gray', game_state.state())
cv2.waitKey()

print(f"Score: %s" % game_state.score())
print(f"Lives: %s" % game_state.lives())

Score: 0
Lives: 3


<a name="6"></a>
## 6 - Deep Q-Learning

<a name="6.1"></a>
### 6.1 Target Network

We can train the $Q$-Network by adjusting it's weights at each iteration to minimize the mean-squared error in the Bellman equation, where the target values are given by:

$$
y = R + \gamma \max_{a'}Q(s',a';w)
$$

where $w$ are the weights of the $Q$-Network. This means that we are adjusting the weights $w$ at each iteration to minimize the following error:

$$
\overbrace{\underbrace{R + \gamma \max_{a'}Q(s',a'; w)}_{\rm {y~target}} - Q(s,a;w)}^{\rm {Error}}
$$

Notice that this forms a problem because the $y$ target is changing on every iteration. Having a constantly moving target can lead to oscillations and instabilities. To avoid this, we can create a separate neural network for generating the $y$ targets. 

We call this separate neural network the **target $\hat Q$-Network** and it will have the same architecture as the original $Q$-Network. By using the target $\hat Q$-Network, the above error becomes:

$$
\overbrace{\underbrace{R + \gamma \max_{a'}\hat{Q}(s',a'; w^-)}_{\rm {y~target}} - Q(s,a;w)}^{\rm {Error}}
$$

where $w^-$ and $w$ are the weights of the target $\hat Q$-Network and $Q$-Network, respectively.

In practice, we will use the following algorithm: every $C$ time steps we will use the $\hat Q$-Network to generate the $y$ targets and update the weights of the target $\hat Q$-Network using the weights of the $Q$-Network. We will update the weights $w^-$ of the the target $\hat Q$-Network using a **soft update**. This means that we will update the weights $w^-$ using the following rule:
 
$$
w^-\leftarrow \tau w + (1 - \tau) w^-
$$

where $\tau\ll 1$. By using the soft update, we are ensuring that the target values, $y$, change slowly, which greatly improves the stability of our learning algorithm.

In [ ]:
# Create the Q-Network
q_network = Sequential([
    Input(shape=state_size), 
    Dense(units=64, activation='relu'), 
    Dense(units=64, activation='relu'), 
    Dense(units=num_actions, activation='linear')
    ])

# Create the target Q^-Network
target_q_network = Sequential([
    Input(shape=state_size),                       
    Dense(units=64, activation='relu'),            
    Dense(units=64, activation='relu'),            
    Dense(units=num_actions, activation='linear')
    ])

optimizer = Adam(learning_rate=ALPHA)


In [ ]:
from classes.number_parser import Number_Parser
import cv2
import random as rng

# 9,9
# Lives 21, 218
# Score 37, 42 - 45 - 53 - 61 - 69 - 77

#matrix = np.asarray(bw_image.getdata(0)).reshape((bw_image.size[1], bw_image.size[0]))

# Convert image to BGR2GRAY format needed by findContours
gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

score_digits = [None] * 6
score_digits[0] = gray[42:42 + 9, 37:37 + 9]
score_digits[1] = gray[42:42 + 9, 45:45 + 9]
score_digits[2] = gray[42:42 + 9, 53:53 + 9]
score_digits[3] = gray[42:42 + 9, 61:61 + 9]
score_digits[4] = gray[42:42 + 9, 69:69 + 9]
score_digits[5] = gray[42:42 + 9, 77:77 + 9]

cv2.imshow('Gray', score_digits[0])
cv2.waitKey()

#print(model.predict(cv2.bitwise_not(score_digits[0])))
#print(model.predict(cv2.bitwise_not(cv2.resize(score_digits[1], (200, 200)))))
#print(model.predict(cv2.bitwise_not(score_digits[2])))
#print(model.predict(cv2.bitwise_not(score_digits[3])))

number_parser = Number_Parser()

print(number_parser.get_number(score_digits))
#print(number_parser.get_digit(score_digits[1]))

"""
number_contours = pickle.load(open('number_contours.dump', 'rb'))
number_contours[0] = score_digits[0]
number_contours[1] = score_digits[1]
number_contours[2] = score_digits[2]
"""


"""
digit_contours, _ = cv2.findContours(score_digits[0], cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
number_contours[0] = digit_contours[0]

digit_contours, _ = cv2.findContours(score_digits[1], cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
number_contours[1] = digit_contours[0]

digit_contours, _ = cv2.findContours(score_digits[2], cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
number_contours[2] = digit_contours[0]
"""

#pickle.dump(number_contours, open('number_contours.dump', 'wb'))

#ret = cv2.matchShapes(score_digits[0], score_digits[3], 1, 0.0)
#print(ret)

#cv2.imshow('Gray', score_digits[2])
#cv2.waitKey()

# Find contours
contours, _ = cv2.findContours(gray, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
centers = [None]*len(contours)
radius = [None]*len(contours)
for i, c in enumerate(contours):
    contours_poly[i] = cv2.approxPolyDP(c, 3, True)
    boundRect[i] = cv2.boundingRect(contours_poly[i])
    centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])

drawing = np.zeros((gray.shape[0], gray.shape[1], 3), dtype=np.uint8)

for i in range(len(contours)):
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    cv2.drawContours(drawing, contours_poly, i, color)
    #cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), \
    #   (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
    #cv2.circle(drawing, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)

cv2.imshow('Gray', drawing)
cv2.waitKey()



In [ ]:
from classes.number_parser import Number_Parser
import cv2
import time

max_num_timesteps = 100

number_parser = Number_Parser()
game.start()

time.sleep(5)

for t in range(max_num_timesteps):
    image = game.get_image_png()

    # Convert image to BGR2GRAY format needed by findContours
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

    score_digits = [None] * 6
    score_digits[0] = gray[42:42 + 9, 37:37 + 9]
    score_digits[1] = gray[42:42 + 9, 45:45 + 9]
    score_digits[2] = gray[42:42 + 9, 53:53 + 9]
    score_digits[3] = gray[42:42 + 9, 61:61 + 9]
    score_digits[4] = gray[42:42 + 9, 69:69 + 9]
    score_digits[5] = gray[42:42 + 9, 77:77 + 9]

    lives_digit = gray[218:218 + 9, 21:21 + 9]

    print(f"Score: %s" % number_parser.get_number(score_digits))
    print(f"Lives: %s" % number_parser.get_digit(lives_digit))
    print("----------")
    time.sleep(1)

In [ ]:
game.fire()

In [31]:
game.quit()